In [0]:
def clean_text(text):
    ''' Lowering text and removing undesirable marks
    '''
    
    text = text.lower()
    text = re.sub(r"\'\n", " ", text)
    text = re.sub(r"\'\xa0", " ", text)
    text = re.sub('\s+', ' ', text) # matches all whitespace characters
    text = text.strip(' ')
    return text


def strip_list_noempty(mylist):
    
    newlist = (item.strip() if hasattr(item, 'strip') else item for item in mylist)
    return [item for item in newlist if item != '']


def clean_punct(text): 
    
    words = token.tokenize(text)
    punctuation_filtered = []
    regex = re.compile('[%s]' % re.escape(punct))
    remove_punctuation = str.maketrans(' ', ' ', punct)
    
    for w in words:
        if w in top_tags:
            punctuation_filtered.append(w)
        else:
            w = re.sub('^[0-9]*', " ", w)
            punctuation_filtered.append(regex.sub('', w))
  
    filtered_list = strip_list_noempty(punctuation_filtered)
        
    return ' '.join(map(str, filtered_list))


def stopWordsRemove(text):
    ''' Removing all the english stop words from a corpus

    Parameter:

    text: corpus to remove stop words from it
    '''

    stop_words = set(stopwords.words("english"))
    words = token.tokenize(text)
    filtered = [w for w in words if not w in stop_words]
    
    return ' '.join(map(str, filtered))


def lemmatization(texts, allowed_postags, stop_words=stop_words):
    ''' It keeps the lemma of the words (lemma is the uninflected form of a word),
    and deletes the underired POS tags
    
    Parameters:
    
    texts (list): text to lemmatize
    allowed_postags (list): list of allowed postags, like NOUN, ADL, VERB, ADV
    '''

    nlp = spacy.load('en', disable=['parser', 'ner'])
    lemma = wordnet.WordNetLemmatizer()       
    doc = nlp(texts) 
    texts_out = []
    
    for token in doc:
        
        if str(token) in top_tags:
            texts_out.append(str(token))
            
        elif token.pos_ in allowed_postags:
            
            if token.lemma_ not in ['-PRON-']:
                texts_out.append(token.lemma_)
                
            else:
                texts_out.append('')
     
    texts_out = ' '.join(texts_out)

    return texts_out


def Recommandation_system_tags(text):
'''  '''
    
    !wget https://www.dropbox.com/s/zr6bpvt6lkr0ew7/P6_data.csv?dl=0

    df = pd.read_csv('P6_data.csv?dl=0')
        # Sampling dataset
    vectorizer_X = TfidfVectorizer(analyzer='word', min_df=0.0, max_df = 1.0, 
                                    strip_accents = None, encoding = 'utf-8', 
                                    preprocessor=None, 
                                    token_pattern=r"(?u)\S\S+", # Need to repeat token pattern
                                    max_features=1000)

    # Binarizing the tags for the supervised models
    multilabel_binarizer = MultiLabelBinarizer()
    y_target = multilabel_binarizer.fit_transform(df['Tags'])

    # 80/20 split
    X_train, X_test, y_train, y_test = train_test_split(
        df['Body'], y_target, test_size=0.2,train_size=0.8, random_state=0)

    # TF-IDF matrices
    X_tfidf_train = vectorizer_X.fit_transform(X_train)
    X_tfidf_test = vectorizer_X.transform(X_test)

    text = clean_text(text)
    text = no_code(text)
    text = clean_punct(text)
    text = stopWordsRemove(text)
    text = lemmatization(text, ['NOUN', 'ADV'])